In [64]:
import ipywidgets as widgets
import requests
from IPython.display import display, clear_output
from bs4 import BeautifulSoup
from lxml import etree
import pandas
import unicodedata

#### Bitte wählen Sie zunächst den gewünschten Katalog: 

In [2]:
auswahl = widgets.Dropdown(
            options=['DNB', 'DMA', 'GND'],  
            value='DNB',
            description='Katalog:',
            disabled=False,
        )

display(auswahl)

Dropdown(description='Katalog:', options=('DNB', 'DMA', 'GND'), value='DNB')

#### Bitte wählen Sie das Metadatenformat für die Ausgabe: 

In [3]:
meta = widgets.Dropdown(
            options=[('MARC21-xml', 'MARC21-xml'), ('DNB Casual (oai_dc)', 'oai_dc'), ('RDF (RDFxml)', 'RDFxml')], 
            value='MARC21-xml',
            description='Metadatenformat:', 
            layout={'width': 'max-content'},
            style={'description_width': 'initial'},
            disabled=False,
        )

display(meta)

Dropdown(description='Metadatenformat:', layout=Layout(width='max-content'), options=(('MARC21-xml', 'MARC21-x…

In [4]:
button = widgets.Button(description="Bestätigen")
output1 = widgets.Output()

display(button, output1)

def on_button_clicked(b):
    global A
    with output1:
        clear_output()
        result = auswahl.value
        if auswahl.value == "DNB":
            selected_url = "https://services.dnb.de/sru/dnb"
        elif auswahl.value == "DMA":
            selected_url = "https://services.dnb.de/sru/dnb.dma"
        elif auswahl.value == "GND":
            selected_url = "https://services.dnb.de/sru/authorities"
        else:
            selected_url = "ERROR: Keine URL gewählt"
        print("Auswahl Katalog-URL für", result, ":", selected_url)
        print("Auswahl Metadatenformat:", meta.value)
    
        A = selected_url
        return A
    
button.on_click(on_button_clicked)


Button(description='Bestätigen', style=ButtonStyle())

Output()

#### Bitte geben Sie nun Ihren Suchbegriff ein: 

In [5]:
searchterm = widgets.Text(
            value='bitte eintippen',
            placeholder='Suchbegriff tippen',
            description='Suchbegriff:',
            disabled=False
        )

display(searchterm)

Text(value='bitte eintippen', description='Suchbegriff:', placeholder='Suchbegriff tippen')

In [62]:
button_search = widgets.Button(description="Suche starten")
output2 = widgets.Output()

display(button_search, output2)

def on_button_clicked(b):
    with output2:
        global records 
        global records_marc
        global r1
        clear_output()
        searchtext = searchterm.value
        print("Suche nach:", searchtext)
        
        
        parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : searchtext, 'recordSchema' : meta.value} 
        r1 = requests.get(A, params = parameter)

        response = BeautifulSoup(r1.content)
        records = response.find_all('record')
        records_marc = response.find_all('record', {'type':'Bibliographic'})
        numberofrecords = response.find_all('numberofrecords')[0].text
        print("Gefundene Treffer:", numberofrecords)
        print("\n", "Vorschau der SRU-Antwort:")
        print(response.prettify()[0:500])
        

button_search.on_click(on_button_clicked)



Button(description='Suche starten', style=ButtonStyle())

Output()

In [126]:
#Funktion für OAI-DC
def parse_record_dc(record):
    
    ns = {"dc": "http://purl.org/dc/elements/1.1/", 
          "xsi": "http://www.w3.org/2001/XMLSchema-instance"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath(".//dc:identifier[@xsi:type='dnb:IDN']", namespaces=ns) #--> Adressiert das Element direkt   
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    #creator:
    creator = xml.xpath('.//dc:creator', namespaces=ns)
    try:
        creator = creator[0].text
    except:
        creator = "N/A"
    
    #titel
    titel = xml.xpath('.//dc:title', namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "N/A"
        
    #date
    date = xml.xpath('.//dc:date', namespaces=ns)
    try:
        date = date[0].text
    except:
        date = "N/A"
    
    
    #publisher
    publ = xml.xpath('.//dc:publisher', namespaces=ns)
    try:
        publ = publ[0].text
    except:
        publ = "N/A"
    
        
    #identifier
    ids = xml.xpath('.//dc:identifier[@xsi:type="tel:ISBN"]', namespaces=ns)
    try:
        ids = ids[0].text
    except:
        ids = "N/A"
        
    #urn
    urn = xml.xpath('.//dc:identifier[@xsi:type="tel:URN"]', namespaces=ns)
    try:
        urn = urn[0].text
    except:
        urn = "N/A"
        
        
    meta_dict = {"idn":idn, "creator":creator, "titel":titel, "date":date, "publisher":publ, "urn":urn, "ISBN":ids}
    
    return meta_dict

In [20]:
#Function für MARC21
def parse_record_marc(item):

    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(item)))
    
    
    #idn
    idn = xml.findall("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'N/A' 
        
    
    #creator
    creator = xml.findall("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        creator = creator[0].text
    except:
        creator = "N/A"
    
    #Titel $a
    title = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    title2 = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'b']", namespaces=ns)
    
    if title and not title2:
        titletext = title[0].text
    elif title and title2:     
        titletext = title[0].text + ": " + title2[0].text
    else:
        titletext = "N/A"
    
    
    #date
    date = xml.findall("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)
    try:
        date = date[0].text
    except:    
        date = 'N/A'
    
    
    #publisher
    publ = xml.findall("marc:datafield[@tag = '264']/marc:subfield[@code = 'b']", namespaces=ns)
    try:
        publ = publ[0].text
    except:    
        publ = 'N/A'
        
        
    #URN
    urn = xml.findall("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    try:
        urn = urn[0].text
    except:    
        urn = 'N/A'
        
        
    #ISBN
    isbn = xml.findall("marc:datafield[@tag = '024']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        isbn = isbn[0].text
    except:    
        isbn = 'N/A'
    

    
    meta_dict = {"idn":idn, "author":creator, "title": titletext, "date":date, 
                 "publisher":publ, "URN":urn, "ISBN":isbn}
    
    return meta_dict
    

#### Speichern der Schnittstellenantwort als XML-Datei: 

In [61]:
button_xml = widgets.Button(description="XML speichern")
output_xml = widgets.Output()

display(button_xml, output_xml)

def on_button_clicked(b):
    with output_xml:
        
        with open('data.xml', 'w', encoding='utf-8') as f:
            print(r1.text, file=f)
    
            
button_xml.on_click(on_button_clicked)

Button(description='XML speichern', style=ButtonStyle())

Output()

#### Aktuell nur für Suchen in den Katalogen DNB und DMA verfügbar: 

In [65]:
button_df = widgets.Button(description="Ausgabe als Tabelle")
output3 = widgets.Output()

display(button_df, output3)

def on_button_clicked(b):
    global df
    with output3:
        clear_output()
        if meta.value == "oai_dc":
            result = [parse_record_dc(record) for record in records]
            df = pandas.DataFrame(result)
            display(df)
        elif meta.value == "MARC21-xml":
            result2 = [parse_record_marc(item) for item in records_marc]
            df = pandas.DataFrame(result2)
            display(df)
        elif meta.value == "RDFxml":
            print("Noch nicht verfügbar")
        else:
            print("ERROR")
            

    
button_df.on_click(on_button_clicked)

Button(description='Ausgabe als Tabelle', style=ButtonStyle())

Output()

#### Tabelle in .csv-Datei überführen: 

In [23]:
button_csv = widgets.Button(description="Als CSV speichern")
output4 = widgets.Output()

display(button_csv, output4)

def on_button_clicked(b):
    with output4:
        df.to_csv("Tabelle.csv")
            
button_csv.on_click(on_button_clicked)

Button(description='Als CSV speichern', style=ButtonStyle())

Output()